In [1]:
import numpy as np
import pandas as pd

from osgeo import gdal, osr
gdal.PushErrorHandler('CPLQuietErrorHandler')
gdal.UseExceptions()
import sys, os, tqdm
from multiprocessing import Pool

In [2]:
file_Input1 ='Mandiri/s1_konawe_selatan.tif'
file_Input2 ='Mandiri/s2_l8_konawe_selatan.tif'
file_location ='Mandiri/regular_points_fin.csv'

In [3]:
print("Input Image 1   :", file_Input1)
print("Input Image 2   :", file_Input2)
    
##File Input 1##
ds2 = gdal.Open(file_Input1)
ulx, xres, xskew, uly, yskew, yres  = ds2.GetGeoTransform()
lrx = ulx + (ds2.RasterXSize * xres)
lry = uly + (ds2.RasterYSize * yres)
gt = ds2.GetGeoTransform()
rows = ds2.RasterYSize
cols = ds2.RasterXSize
jbands = ds2.RasterCount
bands = jbands 
proj_ref = ds2.GetProjectionRef()
training_features = np.zeros((rows*cols,bands),dtype=np.uint16)
for band in range(bands):
    training_features[:,band]= (ds2.GetRasterBand(jbands-bands+band+1).ReadAsArray().astype(np.uint16)).flatten()

Input Image 1   : Mandiri/s1_konawe_selatan.tif
Input Image 2   : Mandiri/s2_l8_konawe_selatan.tif


In [4]:
data_locations = pd.read_csv(file_location)
#print (data_locations)
ndata = len(data_locations)
print ("NData:", ndata);
print ("ulx:", ulx, xres)
print ("uly:", uly, yres)
#data = data_locations.iloc[1,:]
print ("Rows:",rows)
print ("Cols:",cols)
ds2 = None
DNArray = np.zeros((ndata,bands+4),dtype=np.float64)
n=0
for i in range(ndata):
    id = data_locations.iloc[i,0]
    idcls = data_locations.iloc[i,2]
    lon = data_locations.iloc[i,3]
    lat = data_locations.iloc[i,4]
    x= (lon-ulx)/xres
    y= (lat-uly)/yres
    xx = int(x)
    yy = int(y)
    #print (lon, lat, clsnm, xx,yy)
    #DNArray[i,4] = clsnm
    xy=0
    if(xx>-1)and(xx<cols):
        if(yy>-1)and(yy<rows):
            xy = (yy*cols)+xx
            DNArray[n,0] = id
            DNArray[n,1] = idcls
            DNArray[n,2] = lon
            DNArray[n,3] = lat
            DNArray[n,4:] = training_features[xy,:]
            n=n+1
             #print (DNArray[i,:])			 
       #
DNArrayFinal = np.zeros((n,bands+4),dtype=np.float64)
DNArrayFinal[0:n,:] = DNArray[0:n,:]
       #


NData: 3450
ulx: 121.96983567966898 0.0001796630568239043
uly: -3.979896034763128 -0.0001796630568239043
Rows: 3118
Cols: 5233


In [5]:
data_1=pd.DataFrame(DNArrayFinal,columns=['id','Class_ID','xcoord','ycoord','S1_VVmin','S1_VHmin','S1_VVmed','S1_VHmed','S1_VVmax','S1_VHmax'])

In [6]:
##File Input 2#
ds2 = gdal.Open(file_Input2)
ulx, xres, xskew, uly, yskew, yres  = ds2.GetGeoTransform()
lrx = ulx + (ds2.RasterXSize * xres)
lry = uly + (ds2.RasterYSize * yres)
gt = ds2.GetGeoTransform()
rows = ds2.RasterYSize
cols = ds2.RasterXSize
jbands = ds2.RasterCount
bands = jbands 
proj_ref = ds2.GetProjectionRef()
training_features = np.zeros((rows*cols,bands),dtype=np.uint16)
for band in range(bands):
    training_features[:,band]= (ds2.GetRasterBand(jbands-bands+band+1).ReadAsArray().astype(np.uint16)).flatten()

In [7]:
ds2 = None
DNArray = np.zeros((ndata,bands+4),dtype=np.float64)
n=0
for i in range(ndata):
    id = data_locations.iloc[i,0]
    idcls = data_locations.iloc[i,2]
    lon = data_locations.iloc[i,3]
    lat = data_locations.iloc[i,4]
    x= (lon-ulx)/xres
    y= (lat-uly)/yres
    xx = int(x)
    yy = int(y)
    #print (lon, lat, clsnm, xx,yy)
    #DNArray[i,4] = clsnm
    xy=0
    if(xx>-1)and(xx<cols):
        if(yy>-1)and(yy<rows):
            xy = (yy*cols)+xx
            DNArray[n,0] = id
            DNArray[n,1] = idcls
            DNArray[n,2] = lon
            DNArray[n,3] = lat
            DNArray[n,4:] = training_features[xy,:]
            n=n+1
             #print (DNArray[i,:])			 
       #
DNArrayFinal = np.zeros((n,bands+4),dtype=np.float64)
DNArrayFinal[0:n,:] = DNArray[0:n,:]
       #


In [8]:
str_1='l8'
l8_=[]
for i in ['B2','B3','B4','B5','B6']:
    for y in ['min','med','max','mean','stddev']:
        l8_.append(str_1+'_'+i+y)
str_1='s2'
s2_=[]
for i in ['B2','B3','B4','B8','B11']:
    for y in ['min','med','max','mean','stddev']:
        s2_.append(str_1+'_'+i+y)
col_gab=['id','Class_ID','xcoord','ycoord']+l8_+s2_

In [9]:
data_2=pd.DataFrame(DNArrayFinal,columns=col_gab)[l8_+s2_]

In [10]:
gab_data_=data_1.merge(data_2,left_index=True,right_index=True)

In [11]:
gab_data_.to_csv('Mandiri/data_latihan_train.csv',sep=',',index=False)